In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import base

import warnings
warnings.filterwarnings('ignore')

In [426]:
#https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/

In [2]:
training_data = pd.read_csv("../data/TrainingSet.csv",index_col=0)
submission_labels = pd.read_csv("../data/SubmissionRows.csv", index_col=0)


In [3]:
#df = training_data.melt(id_vars=['Series Code', 'Country Name','Series Name'], var_name='Year', value_name='indicator',ignore_index = False)
df = training_data.melt(id_vars=['Series Code', 'Country Name'], var_name='Year', value_name='indicator',ignore_index = False, value_vars=['1972 [YR1972]','1973 [YR1973]','1974 [YR1974]','1975 [YR1975]',\
                                                                                                                                          '1976 [YR1976]','1977 [YR1977]','1978 [YR1978]','1979 [YR1979]', \
                                                                                                                                          '1980 [YR1980]',  '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]', \
                                                                                                                                          '1984 [YR1984]','1985 [YR1985]','1986 [YR1986]','1987 [YR1987]', \
                                                                                                                                          '1988 [YR1988]','1989 [YR1989]','1990 [YR1990]','1991 [YR1991]', \
                                                                                                                                          '1992 [YR1992]','1993 [YR1993]','1994 [YR1994]','1995 [YR1995]', \
                                                                                                                                          '1996 [YR1996]', '1997 [YR1997]','1998 [YR1998]','1999 [YR1999]', \
                                                                                                                                          '2000 [YR2000]','2001 [YR2001]','2002 [YR2002]','2003 [YR2003]' , \
                                                                                                                                          '2004 [YR2004]','2005 [YR2005]','2006 [YR2006]','2007 [YR2007]'])

df['Year'] = df['Year'].str.extract('(\d+)', expand=False) #.4stype(i4t),'2005 [YR2005]','2006 [YR2006]'

#df = df.sort_values(['Week', 'Product_Code'])

pd.set_option('display.max_colwidth', -1)

df['Year'] = pd.to_datetime(df['Year'], format='%Y')

df.fillna(0, inplace=True)

df.head()


,Series Code,Country Name,Year,indicator
0,allsi.bi_q1,Afghanistan,1972-01-01,0.0
1,allsp.bi_q1,Afghanistan,1972-01-01,0.0
2,allsa.bi_q1,Afghanistan,1972-01-01,0.0
4,allsi.gen_pop,Afghanistan,1972-01-01,0.0
5,allsp.gen_pop,Afghanistan,1972-01-01,0.0


In [4]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime
import re
import time
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
#import pydot_ng as pydot
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import plot_model
from keras.optimizers import SGD
from sklearn.model_selection import KFold, cross_val_score

In [29]:
# target Encoding Does not Work 
'''
target_encode_columns = ['Country Name','Series Code']
target = ['indicator']
target_encode_df = df[target_encode_columns + target] #.reset_index().drop(columns = 'index', axis = 1)
target_name = target[0]
target_df = pd.DataFrame()
for embed_col in target_encode_columns:
    val_map = target_encode_df.groupby(embed_col)[target].mean().to_dict()[target_name]
    target_df[embed_col] = target_encode_df[embed_col].map(val_map).values
    
score_target_drop = df.drop(target_encode_columns, axis = 1).reset_index().drop(columns = 'index', axis = 1)
score_target = pd.concat([score_target_drop, target_df], axis = 1)

score_target['Year']=score_target['Year'].dt.year
score_target.head()
'''


# Label Encoding 

encode_columns = ['Country Name','Series Code']
encode_df = df[encode_columns]
encode_df = encode_df.astype('str')
encode_df = encode_df.apply(LabelEncoder().fit_transform)
score_encode_drop = df.drop(encode_columns, axis = 1)
#score_encode_drop=df

score_encode = pd.concat([score_encode_drop, encode_df], axis = 1)
score_encode['Year']=df['Year'].dt.year
score_target=score_encode


In [19]:


#score_target.groupby('Country Name').nunique()

In [30]:
score_encode.head()

,Year,indicator,Country Name,Series Code
0,1972,0.0,0,1296
1,1972,0.0,0,1299
2,1972,0.0,0,1293
4,1972,0.0,0,1298
5,1972,0.0,0,1301


In [31]:
X_train, X_test,y_train,y_test = train_test_split(score_target[['Year','Country Name','Series Code']],score_target['indicator'], test_size=0.5, random_state=42)


In [54]:
#https://www.kaggle.com/robikscube/tutorial-time-series-forecasting-with-xgboost
#not working

In [32]:
print(X_train.iloc[559])

Year            2001
Country Name    166 
Series Code     1011
Name: 222451, dtype: int64


In [28]:

params = {'colsample_bytree': uniform(0.7, 0.3),
          'gamma': uniform(0, 0.5),
          'learning_rate': uniform(0.003, 0.1), # default 0.1 
          'max_depth': randint(2, 6), # default 3
          'n_estimators': randint(100, 250), # default 100
          'subsample': uniform(0.6, 0.4)}
xgb_model = xgb.XGBRegressor(objective="reg:linear",random_state=42)
time_split = TimeSeriesSplit(n_splits = 4)
#time_split = KFold(n_splits = 3)
#xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=2, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=42, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
x_model=xgb_search.fit(X_train,y_train)

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[20:10:56] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:11:07] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:11:27] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:01] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:48] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:13:18] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:14:24] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:16:10] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:sq

In [34]:
y_pred = xgb_search.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print ('RMSE:', rms)

RMSE: 13283906204650.55


In [35]:
# connect predictions with outputs
for i in range(len(y_pred)):
	print(X_test.loc[i], "pred is: " + str(y_pred[i]))
    

   Year  Country Name  Series Code
0  1972  0             1296       
0  2002  0             1296       
0  1992  0             1296       
0  2004  0             1296       
0  1974  0             1296       
0  2003  0             1296       
0  1980  0             1296       
0  1977  0             1296       
0  1996  0             1296       
0  1997  0             1296       
0  1988  0             1296       
0  2000  0             1296       
0  1982  0             1296       
0  2005  0             1296       
0  1991  0             1296       
0  1999  0             1296       
0  1989  0             1296        pred is: -92444205000.0
   Year  Country Name  Series Code
1  2000  0             1299       
1  1998  0             1299       
1  1973  0             1299       
1  1995  0             1299       
1  1999  0             1299       
1  1988  0             1299       
1  1977  0             1299       
1  1986  0             1299       
1  1975  0             1299    

KeyError: 3

In [36]:
print(X_test.iloc[559],y_pred[559])

Year            1993
Country Name    206 
Series Code     233 
Name: 276370, dtype: int64 55320895000.0


In [75]:
submission_results=submission_labels.join(training_data)

submission_results=submission_results[['Country Name','Series Code']]

submission_results['Year']=2008

submission_results['indicator']=0

#submission_results.head() 

submission_results.groupby('Year').nunique()

,Country Name,Series Code,indicator
Year,,,
2008,206,7,1


In [ ]:
# Target Encoding does not work 

In [77]:
'''target_encode_columns = ['Country Name','Series Code']
target = ['indicator']
target_encode_df = submission_results[target_encode_columns + target] #.reset_index().drop(columns = 'index', axis = 1)
target_name = target[0]
target_df = pd.DataFrame()
for embed_col in target_encode_columns:
    val_map = target_encode_df.groupby(embed_col)[target].mean().to_dict()[target_name]
    target_df[embed_col] = target_encode_df[embed_col].map(val_map).values
    
score_predict_drop = submission_results.drop(target_encode_columns, axis = 1).reset_index().drop(columns = 'index', axis = 1)
score_predict = pd.concat([score_target_drop, target_df], axis = 1)

score_predict['Year']= pd.to_datetime(score_target['Year'], format='%Y').dt.year
'''

# Label Encoding 

encode_columns = ['Country Name','Series Code']
pred_df = submission_results[encode_columns]
score_predict_drop = submission_results.drop(encode_columns, axis = 1)
pred_df = pred_df.astype('str')
pred_df = pred_df.apply(LabelEncoder().fit_transform)


score_predict_2008 = pd.concat([score_predict_drop, pred_df], axis = 1)

score_predict['Year']=pd.to_datetime(score_predict_2008['Year'],format='%Y').dt.year

res_pred_01 = xgb_search.predict(score_predict_2008[['Year', 'Country Name', 'Series Code']])

score_predict_2008['predictions'] = res_pred_01

score_predict_2008.head()



,Year,indicator,Country Name,Series Code,predictions
559,2008,0,0,5,-8.041746e+11
618,2008,0,0,6,-8.041746e+11
753,2008,0,0,1,-8.041746e+11
1030,2008,0,0,3,-8.041746e+11
1896,2008,0,1,5,-8.041746e+11


In [78]:
submission_results=submission_labels.join(training_data)

submission_results=submission_results[['Country Name','Series Code']]

submission_results['Year']=2009

submission_results['indicator']=0

# Label Encoding 

encode_columns = ['Country Name','Series Code']
pred_df = submission_results[encode_columns]
score_predict_drop = submission_results.drop(encode_columns, axis = 1)
pred_df = pred_df.astype('str')
pred_df = pred_df.apply(LabelEncoder().fit_transform)


score_predict_2009 = pd.concat([score_predict_drop, pred_df], axis = 1)

score_predict_2009['Year']=pd.to_datetime(score_predict_2009['Year'],format='%Y').dt.year

res_pred_01 = xgb_search.predict(score_predict_2009[['Year', 'Country Name', 'Series Code']])

score_predict_2009['predictions'] = res_pred_01

score_predict_2009.head()



,Year,indicator,Country Name,Series Code,predictions
559,2009,0,0,5,-8.041746e+11
618,2009,0,0,6,-8.041746e+11
753,2009,0,0,1,-8.041746e+11
1030,2009,0,0,3,-8.041746e+11
1896,2009,0,1,5,-8.041746e+11


In [97]:
predictions_from_model=score_predict_2009.join(score_predict_2008,how='inner',lsuffix=2009, rsuffix=2008)


predictions_from_model=predictions_from_model.drop(columns=['Year2009','indicator2009','Country Name2009','Year2008','indicator2008','Country Name2008','Series Code2009','Series Code2008'])


predictions_from_model=predictions_from_model.rename(columns={"predictions2008": "2008 [YR2008]", "predictions2009": "2009 [YR2009]"})

predictions_from_model.to_csv("../data/result_01.csv")
